# New Segment model

Testing adding functions to the body model using the new functions 

In [1]:
import NewSegment as ns
import numpy as np

In [2]:
# now we want to create some methods 
# for building the matrix and the RHS

def vapor_pressure(temp):
    """Compute vapor pressure of water for a given temperature
    using Antoine Equation using parameters from DDB
    This parameterization is valid between 1 and 100C
    This returns pressure in mmHg
    """
    A = 8.07131
    B = 1730.63
    C = 233.426
    power = A - B / (C + temp)
    return 10 ** power 

def airvaporpressure(tair, rhumid):
    """Convert the air temperature and relative humidity to
    a vapor pressure
    """
    vpmax = vapor_pressure(tair)
    return vpmax * rhumid

def emax(tskin, tair, rhumid, covcoeff, sarea):
    """return emax for a segment given it's temperature, the
    vapor pressure in air, the convetive heat coefficent, and the
    surface area
    
    This assumes that the air velocity is 0.1 m/s 
    (natural convection)
    
    need to get the vapor pressure in the air from the relative
    humidity and air temperature
    """
    # lewis relation and conversion between kcal/hr to W
    lewis_relation = 2.2 * 1.162
    pair = airvaporpressure(tair, rhumid)
    return (vapor_pressure(tskin) - pair) \
            * lewis_relation * covcoeff * sarea

def ebody(node, tenv, rhumid, bodys):
    """For a node, compute the evaporative heat loss in Watts
    """
    skins = node["skins"]
    sweat = bodys.bodyvalues["sweat"]
    ebase = node["ebase"]
    error = node["temp"] - node["tset"]
    bullconst = 10 # degrees c for local effect
    local = 2.0 ** (error / bullconst)
    esweat = (ebase + skins * sweat) * local
    etop = emax(node["temp"], tenv, rhumid, node["hconv"], node["sarea"])
    return min(esweat, etop)
    

In [3]:
emax(20, 23, 0.8, 0.5, 3)

2.5745176337923423

In [4]:
# functions for the body parameters based on each of the 
# nodes

def errors(tree, bodyvals):
    tsets = np.array(tree.get_param("tset"))
    temps = np.array(tree.get_param("temp"))
    return temps - tsets

def wrms(tree, bodyvals):
    # positive or zero errors
    err = errors(tree, bodyvals)
    wrm = np.clip(err, 0, None)
    return np.sum(wrm * np.array(tree.get_param("skinr")))

def clds(tree, bodyvals):
    # negative or zero erros 
    err = errors(tree, bodyvals)
    cld = -np.clip(err, None, 0)
    return np.sum(cld * np.array(tree.get_param("skinr")))

def sweat(tree, bodyvals):
    # should be called after the wrms adn clds
    # assume that the core is the head node of the tree
    coreerr = tree["temp"] - tree["tset"]
    cws = 371.2 # W/oC
    sswt = 33.6 # W/oC
    wrms = bodyvals["wrms"]
    clds = bodyvals["clds"]
    return cws * coreerr + sswt * (wrms - clds)


In [5]:

def melems(node, mat, dt, tenv, rhumid, bodys):
    nidx = node.idx() # the row that we are going to be working with
    # handel the Parent node (proximal blood flow)
    if node.parent is not None:
        bfcp = node.parent.lnkprms[node]["bfc"]
        mat[nidx,node.parent.idx()] = bfcp
    else:
        bfcp = 0
    # handel the Chilren nodes
    bfcct = 0 # total children blood flow coefficent
    for child in node.children:
        bfcc = node.lnkprms[child]["bfc"]
        mat[nidx,child.idx()] = bfcc
        bfcct += bfcc
    # and then the self term
    mat[nidx,nidx] = -node["hcomb"] - bfcp \
                     - bfcct - node["capacity"] / dt
        
def rhelems(node, vec, dt, tenv, rhumid, bodys):
    nidx = node.idx()
    q = node["qbase"]
    c = node["capacity"]
    e = ebody(node, tenv, rhumid, bodys)
    hcomb = node["hcomb"]
    node[nidx] = q - (c / dt) + e + (hcomb * tenv)



In [6]:
# and then build the tree of body segments and
# add in the parameters, and run simulations
torso = ns.BodyNode("Torso")
torso.add_child(ns.BodyNode("Head"), {"bfc": 51.70})
torso.add_child(ns.BodyNode("LUArm").add_child(
                    ns.BodyNode("LFArm").add_child(
                        ns.BodyNode("LHand"), 
                        {"bfc": 1.20}), 
                    {"bfc": 2.65}), 
                {"bfc": 5.45})
torso.add_child(ns.BodyNode("RUArm").add_child(
                    ns.BodyNode("RFArm").add_child(
                        ns.BodyNode("RHand"), 
                        {"bfc": 1.20}), 
                    {"bfc": 2.65}), 
                {"bfc": 5.45})
torso.add_child(ns.BodyNode("LULeg").add_child(
                    ns.BodyNode("LLLeg").add_child(
                        ns.BodyNode("LFoot"), 
                        {"bfc": 0.56}), 
                    {"bfc": 0.85}), 
                {"bfc": 2.72})
torso.add_child(ns.BodyNode("RULeg").add_child(
                    ns.BodyNode("RLLeg").add_child(
                        ns.BodyNode("RFoot"), 
                        {"bfc": 0.56}), 
                    {"bfc": 0.85}), 
                {"bfc": 2.72})

# Parameters are mostly guessed from the tanabe and the stolwijk
# paper
# and then set the parameters on each of the nodes
# an initial temperature
torso.set_param("temp", [35.0, 36.1, 34.4, 34.9, 
                          35.3, 34.4, 34.9, 35.3, 
                          34.8, 34.3, 34.6, 34.8, 
                          34.3, 34.6])
# heat capacity
torso.set_param("capacity", [171380.0, 17556.0, 9041.34, 5739.14, 
                             1400.3, 9041.34, 5739.14, 1400.3,
                             29314.3, 13973.7, 2006.4, 29314.3,
                             13973.7, 2006.4])
# base metabolic rate
torso.set_param("qbase", [59.539, 17.3, 1.264, 0.371,
                          0.140, 1.264, 0.371, 0.140,
                          1.440, 0.380, 0.313, 1.440,
                          0.380, 0.313])
# convective h
torso.set_param("hconv", [2.498, 3.196, 3.486, 3.486,
                          3.893, 3.486, 3.486, 3.893,
                          3.196, 3.196, 3.486, 3.196,
                          3.196, 3.486])
# combined h
torso.set_param("hcomb", [7.727, 9.587, 8.483, 8.483,
                          7.379, 8.483, 8.483, 7.379,
                          7.844, 7.844, 8.134, 7.844,
                          7.844, 8.134])
# surface area
torso.set_param("sarea", [0.557, 0.140, 0.096, 0.063, 
                          0.050, 0.096, 0.063, 0.050, 
                          0.209, 0.112, 0.056, 0.209, 
                          0.112, 0.056])
# skins
torso.set_param("skins", [0.481, 0.081, 0.051, 0.026, 
                          0.016, 0.051, 0.026, 0.016,
                          0.073, 0.036, 0.018, 0.073,
                          0.036, 0.018])
# skinr
torso.set_param("skinr", [0.493, 0.070, 0.023, 0.012,
                          0.092, 0.023, 0.012, 0.092,
                          0.050, 0.025, 0.017, 0.050,
                          0.025, 0.017])
# tset
torso.set_param("tset", [35.0, 36.1, 34.4, 34.9, 
                         35.3, 34.4, 34.9, 35.3, 
                         34.8, 34.3, 34.6, 34.8, 
                         34.3, 34.6])


In [7]:
# and set all the functions 
def set_mat_rhs_funcs(node):
    node.matf = melems
    node.vecf = rhelems
    
torso._walk(set_mat_rhs_funcs)

In [8]:
# create the body
body = ns.Body(torso)

In [9]:
# set the body update functions
body.register_body_parameter(pfunc=wrms, pname="wrms", pstart=0)
body.register_body_parameter(pfunc=clds, pname="clds", pstart=0)
body.register_body_parameter(pfunc=sweat, pname="sweat", pstart=0)

In [10]:
body.register_log_parameter("temp", "tree")
body.register_log_parameter("clds", "body")
body.register_log_parameter("wrms", "body")

In [11]:
body.run_constant_temp(0.1, [25, 0.25], 10000)

{'temp': [[35.0,
   36.1,
   34.4,
   34.9,
   35.3,
   34.4,
   34.9,
   35.3,
   34.8,
   34.3,
   34.6,
   34.8,
   34.3,
   34.6],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [12]:
# Humm. That looks like that didnt work